In [4]:
%%writefile -a proba.py

%load_ext tensorboard

import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import DepthwiseConv2D

from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import TensorBoard

absProjectDir = Path(os.getcwd()).resolve().parents[0]
projectDir = os.path.relpath(absProjectDir,os.curdir)
load_dotenv(find_dotenv())

trainDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TRAIN_DF"))
testDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TEST_DF"))
testOrigDir = os.path.join(projectDir, os.environ.get("PROC_TEST_ORIG_DIR"))
testAugmDir = os.path.join(projectDir, os.environ.get("PROC_TEST_AUG_DIR"))
trainOrigDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_ORIG_DIR"))
trainAugmDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_AUG_DIR"))
testRootDir = os.path.commonpath([testOrigDir, testAugmDir])
trainRootDir = os.path.commonpath([trainOrigDir, trainAugmDir])
modelDir = os.path.join(projectDir, os.environ.get("MODEL_DIR"))

sys.path.append(os.path.join(projectDir,'src/data'))
sys.path.append(os.path.join(projectDir,'src/models'))
sys.path.append(os.path.join(projectDir,'src/visualization'))
import imggen as imgen
from customlayer import DepthWiseConvBatchNorm
from customlayer import ConvBatchNorm
from customlayer import InceptionType
from customlayer import CustomInceptionFactory

from basemodel import BaseModel

from models import DepthWiseConvModel1
from models import ConvModel1
from models import InterceptionModelV1
from models import InterceptionModelV2
from models import StemDecorator
from models import StemDecoratorConv
from models import StemDecoratorDepthWiseConv
from models import PredefinedModel

from visualize import VisualizeIntermedianActivation #(layer_cnt, model, img)
from visualize import GetActivatedRegion #(model, layer_name, img, activator = 0)
from visualize import CAM

from tensorflow.keras.applications import InceptionResNetV2

Appending to proba.py


0 - (1443, 362)
1 - (295, 75)
2 - (799, 200)
3 - (154, 39)
4 - (235, 60)

In [3]:
%%writefile proba.py

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionResNetV2

Writing proba.py


In [3]:
def SaveModel(model,name,modelDir):
    path = os.path.join(modelDir,name)
    for suf in [''] + [str(i) for i in range(256)]:
        modified_path = path + suf
        if not os.path.exists(modified_path):
            os.makedirs(modified_path)
            path = modified_path
            break
    model.save(path)
    return path

In [4]:
trgen = imgen.Generator(trainDfPath, trainRootDir, (224,224))
trgen.SetDataFrameForGeneration([[3],[4]],[[1024],[1024]])
train_set = trgen.GetGenerator(bs = 64)

tsgen = imgen.Generator(testDfPath,testRootDir, (224,224))
tsgen.SetDataFrameForGeneration([[3],[4]], [[1024],[1024]])
test_set = tsgen.GetGenerator(bs = 128)

'''evalgen = imgen.Generator(testDfPath,testRootDir, (224,224))
evalgen.SetDataFrameForGeneration([[2],[3]], [[512],[512]])
eval_set = tsgen.GetGenerator(bs = 1024)'''

Found 2048 validated image filenames belonging to 2 classes.
Found 2048 validated image filenames belonging to 2 classes.


'evalgen = imgen.Generator(testDfPath,testRootDir, (224,224))\nevalgen.SetDataFrameForGeneration([[2],[3]], [[512],[512]])\neval_set = tsgen.GetGenerator(bs = 1024)'

In [5]:

class PredefinedModel(BaseModel):
    def __init__(self, model, top_layers = 1, intercept_module = BaseModel()):
        super(PredefinedModel,self).__init__(intercept_module.__dict__)
        self.top_layers = top_layers
        self.pred_model = model

    def getDescription(self):
        return self.pred_model.name + "_" + super(PredefinedModel,self).getDescription()

    def build_core_part_of_model(self, model):
        for m in self.pred_model.layers:
            m.trainable = True
        for i in range(len(self.pred_model.layers) - self.top_layers):
            self.pred_model.layers[i].trainable = False
        model.add(self.pred_model)
        return model

In [8]:
basemodel = BaseModel(learn_rate = 0.001, L2 = 0.0001, top_layer_cnt = 2, units = 512)
IncResNetV2 = InceptionResNetV2(include_top=False, weights='imagenet')

Model inception_resnet_v2_PredefinedModel_F{'filters': 32, 'units': 512, 'input_shape': (224, 224, 3), 'learn_rate': 0.001, 'l1': None, 'l2': 0.0001, 'drop': None, 'conv_layer_cnt': 6, 'top_layer_cnt': 2, 'model': None}_U512_LR0001_LCnt6 is built


In [13]:
IncResNetV2_model = PredefinedModel(model = IncResNetV2, top_layers = 5, intercept_module = basemodel).Create()

Model inception_resnet_v2_PredefinedModel_F{'filters': 32, 'units': 512, 'input_shape': (224, 224, 3), 'learn_rate': 0.001, 'l1': None, 'l2': 0.0001, 'drop': None, 'conv_layer_cnt': 6, 'top_layer_cnt': 2, 'model': None}_U512_LR0001_LCnt6 is built


In [18]:
for ss in basemodel.__dict__:
    print(ss)

filters
units
input_shape
learn_rate
l1
l2
drop
conv_layer_cnt
top_layer_cnt
model


In [23]:
for key, val in basemodel.__dict__.items():
    if hasattr(IncResNetV2_model, key):
        setattr(IncResNetV2_model, key,val)

In [24]:
IncResNetV2_model.__dict__

{'filters': 32,
 'units': 512,
 'input_shape': (224, 224, 3),
 'learn_rate': 0.001,
 'l1': None,
 'l2': 0.0001,
 'drop': None,
 'conv_layer_cnt': 6,
 'top_layer_cnt': 2,
 'model': None,
 'top_layers': 5,
 'pred_model': <tensorflow.python.keras.engine.training.Model at 0x1229dbf6fd0>}

# Inception ResNet V2

# 1.

In [6]:
EPOCH = 120
basemodel = BaseModel(learn_rate = 0.001)
train_cnt = 512
test_cnt = 1024
IncResNetV2 = InceptionResNetV2(include_top=False, weights='imagenet')

In [15]:
def Scheduler1(epoch):
    if epoch < 20:
        return 0.001
    if epoch <  40:
        return 0.00065
    if epoch <  60:
        return 0.0003
    return 0.0001

In [16]:
IncResNetV2_model = PredefinedModel(model = IncResNetV2, top_layers = 5, intercept_module = basemodel).Create()

Model inception_resnet_v2_BaseModel_F32_U512_LR001_LCnt6 is built


In [18]:
extra_callbacks = [TensorBoard(log_dir="logs", histogram_freq=5)]

In [ ]:
IncResNetV2_model.fit(train_set = train_set, val_set = test_set, epochs = EPOCH, shedule = Scheduler1,
                log = True, threshold = 0.7,extra_callbacks = extra_callbacks)

SUMUP: After 22 epoch the training loss is very close to 0 but the test loss is about 13 that definetly overfitting. So i think in the next experiment some regularization should be added. As well as play with number of training example, when it increased, the generalization should be increased too, i suggest..

# 2.

In [7]:
EPOCH = 120
basemodel = BaseModel(learn_rate = 0.001, L2 = 0.0001, top_layer_cnt = 2, units = 512)
train_cnt = 1024
test_cnt = 1024
IncResNetV2 = InceptionResNetV2(include_top=False, weights='imagenet')
IncResNetV2_model = PredefinedModel(model = IncResNetV2, top_layers = 5, intercept_module = basemodel).Create()
extra_callbacks = [TensorBoard(log_dir="logs", histogram_freq=5)]
def Scheduler1(epoch):
    if epoch < 10:
        return 0.001
    if epoch <  30:
        return 0.00065
    if epoch <  50:
        return 0.0003
    return 0.0001

Model inception_resnet_v2_PredefinedModel_F{'filters': 32, 'units': 512, 'input_shape': (224, 224, 3), 'learn_rate': 0.001, 'l1': None, 'l2': 0.0001, 'drop': None, 'conv_layer_cnt': 6, 'top_layer_cnt': 2, 'model': None}_U512_LR0001_LCnt6 is built


In [7]:
IncResNetV2_model.fit(train_set = train_set, val_set = test_set, epochs = EPOCH, shedule = Scheduler1,
                log = True, threshold = 0.7,extra_callbacks = extra_callbacks)

Train for 32 steps, validate for 16 steps
Epoch 1/120
32/32 [==============================] - 1024s 32s/step - loss: 0.4099 - accuracy: 0.8154 - val_loss: 2.3128 - val_accuracy: 0.5664
Epoch 2/120
32/32 [==============================] - 929s 29s/step - loss: 0.0829 - accuracy: 0.9785 - val_loss: 10.2990 - val_accuracy: 0.5000
Epoch 3/120
32/32 [==============================] - 938s 29s/step - loss: 0.0549 - accuracy: 0.9844 - val_loss: 5.5073 - val_accuracy: 0.5400
Epoch 4/120
32/32 [==============================] - 929s 29s/step - loss: 0.0210 - accuracy: 0.9917 - val_loss: 8.7228 - val_accuracy: 0.5234
Epoch 5/120
32/32 [==============================] - 893s 28s/step - loss: 0.0275 - accuracy: 0.9922 - val_loss: 6.5899 - val_accuracy: 0.4565
Epoch 6/120
32/32 [==============================] - 922s 29s/step - loss: 0.0274 - accuracy: 0.9907 - val_loss: 13.0229 - val_accuracy: 0.5249
Epoch 7/120
32/32 [==============================] - 926s 29s/step - loss: 0.0188 - accuracy: 0.9

KeyError: 'val_accuracy'

In [9]:
super(PredefinedModel,IncResNetV2_model).getDescription()

"PredefinedModel_F{'filters': 32, 'units': 512, 'input_shape': (224, 224, 3), 'learn_rate': 0.001, 'l1': None, 'l2': 0.0001, 'drop': None, 'conv_layer_cnt': 6, 'top_layer_cnt': 2, 'model': None}_U512_LR0001_LCnt6"

In [10]:
IncResNetV2_model.__class__.__mro__

(models.PredefinedModel, basemodel.BaseModel, object)

In [17]:
for layer in IncResNetV2_model.model.layers:
    print(layer.kernel_regularizer)

AttributeError: 'Model' object has no attribute 'kernel_regularizer'

In [16]:
IncResNetV2_model.model.optimizer.l2

AttributeError: 'Adam' object has no attribute 'l2'

In [18]:
IncResNetV2_model.__dict__

{'filters': {'filters': 32,
  'units': 512,
  'input_shape': (224, 224, 3),
  'learn_rate': 0.001,
  'l1': None,
  'l2': 0.0001,
  'drop': None,
  'conv_layer_cnt': 6,
  'top_layer_cnt': 2,
  'model': None},
 'units': 512,
 'input_shape': (224, 224, 3),
 'learn_rate': 0.0001,
 'l1': None,
 'l2': None,
 'drop': None,
 'conv_layer_cnt': 6,
 'top_layer_cnt': 1,
 'model': <tensorflow.python.keras.engine.sequential.Sequential at 0x2370b7fc748>,
 'top_layers': 5,
 'pred_model': <tensorflow.python.keras.engine.training.Model at 0x23733468320>}

In [6]:
IncResNetV2_model.model.summary()

Model: "inception_resnet_v2_PredefinedModel_F{'filters': 32, 'units': 512, 'input_shape': (224, 224, 3), 'learn_rate': 0.001, 'l1': None, 'l2': 0.0001, 'drop': None, 'conv_layer_cnt': 6, 'top_layer_cnt': 2, 'model': None}_U512_LR0001_LCnt6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  multiple                  54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 55,124,193
Trainable params: 4,917,793
Non-trainable params: 50,206,400
_________________________________________________________________


# VGG16 

In [5]:
EPOCH = 120
basemodel = BaseModel(learn_rate = 0.001)
vgg16 = VGG16(include_top=False, weights='imagenet')  

In [6]:
def Scheduler1(epoch):
    if epoch < 20:
        return 0.001
    if epoch <  40:
        return 0.00065
    if epoch <  60:
        return 0.0003
    return 0.0001

## 1. Top Layers = 4

In [7]:
vgg16_model = PredefinedModel(model = vgg16, top_layers = 4, intercept_module = basemodel).Create()

Model vgg16_BaseModel_F32_U512_LR001_LCnt6 is built


In [8]:
extra_callbacks = [TensorBoard(log_dir="logs", histogram_freq=5)]

In [ ]:
vgg16_model.fit(train_set = train_set, val_set = test_set, epochs = EPOCH, shedule = Scheduler1,
                log = True, threshold = 0.7,extra_callbacks = extra_callbacks)

I interapt the training on the 10 epoch because it is definetly inefficient model, and take a greate amount of time.
Here i used 2048 train cases and 1024 test cases per category.

In [40]:
mobNetV2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),alpha=1.0, include_top=False)

In [49]:
for i in range(len(mobNetV2.layers) - 10):
    mobNetV2.layers[i].trainable = False

In [50]:
inputs = tf.keras.Input(shape=(224,224,3), name = "My_func_Model")
x = mobNetV2(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
modelNetV2 = Model(inputs = inputs, outputs = outputs )

In [51]:
modelNetV2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
My_func_Model (InputLayer)   [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1280)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               655872    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       
Total params: 2,914,369
Trainable params: 1,388,865
Non-trainable params: 1,525,504
_________________________________________________________________


In [54]:
%load_ext tensorboard

In [56]:
modelNetV2.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])

In [57]:
log_dir = "logs"# + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history3 = modelNetV2.fit(train_set,epochs = 20, validation_data = test_set, callbacks=[tensorboard_callback])

Train for 64 steps, validate for 64 steps
Epoch 1/20
64/64 [==============================] - 360s 6s/step - loss: 0.6064 - accuracy: 0.6675 - val_loss: 0.6837 - val_accuracy: 0.5698
Epoch 2/20
64/64 [==============================] - 343s 5s/step - loss: 0.4164 - accuracy: 0.8223 - val_loss: 0.7727 - val_accuracy: 0.5020
Epoch 3/20
64/64 [==============================] - 340s 5s/step - loss: 0.3068 - accuracy: 0.8989 - val_loss: 1.1526 - val_accuracy: 0.4917
Epoch 4/20
64/64 [==============================] - 335s 5s/step - loss: 0.2166 - accuracy: 0.9443 - val_loss: 1.1212 - val_accuracy: 0.4966
Epoch 5/20
64/64 [==============================] - 326s 5s/step - loss: 0.1484 - accuracy: 0.9753 - val_loss: 1.3181 - val_accuracy: 0.5034
Epoch 6/20
64/64 [==============================] - 327s 5s/step - loss: 0.0949 - accuracy: 0.9917 - val_loss: 2.4057 - val_accuracy: 0.4980
Epoch 7/20
64/64 [==============================] - 326s 5s/step - loss: 0.0619 - accuracy: 0.9983 - val_loss: 2

In [60]:
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 1600.

In [58]:
inceptV3 = tf.keras.applications.InceptionV3(include_top=False, input_shape=(224,224,3))

87916544/87910968 [==============================] - 6s 0us/step


In [59]:
inceptV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 111, 111, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 111, 111, 32) 96          conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization_12[0][0]     
_______________________________________________________________________________________

In [61]:
for i in range(len(inceptV3.layers) - 10):
    inceptV3.layers[i].trainable = False

In [62]:
inputs = tf.keras.Input(shape=(224,224,3), name = "My_InceptV3")
x = inceptV3(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
modelInceptV3 = Model(inputs = inputs, outputs = outputs )
modelInceptV3.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])

In [66]:
log_dir = "logs2"# + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history4 = modelInceptV3.fit(train_set,epochs = 20, validation_data = test_set, callbacks=[tensorboard_callback])

Train for 64 steps, validate for 64 steps
Epoch 1/20
64/64 [==============================] - 578s 9s/step - loss: 0.6587 - accuracy: 0.6047 - val_loss: 0.7885 - val_accuracy: 0.5718
Epoch 2/20
64/64 [==============================] - 534s 8s/step - loss: 0.5892 - accuracy: 0.6924 - val_loss: 0.6938 - val_accuracy: 0.5786
Epoch 3/20
64/64 [==============================] - 513s 8s/step - loss: 0.5420 - accuracy: 0.7312 - val_loss: 0.7316 - val_accuracy: 0.5820
Epoch 4/20
64/64 [==============================] - 519s 8s/step - loss: 0.5121 - accuracy: 0.7634 - val_loss: 0.7179 - val_accuracy: 0.5835
Epoch 5/20
64/64 [==============================] - 511s 8s/step - loss: 0.4915 - accuracy: 0.7720 - val_loss: 0.6889 - val_accuracy: 0.6147
Epoch 6/20
64/64 [==============================] - 523s 8s/step - loss: 0.4659 - accuracy: 0.7930 - val_loss: 0.7107 - val_accuracy: 0.6001
Epoch 7/20
64/64 [==============================] - 505s 8s/step - loss: 0.4482 - accuracy: 0.8027 - val_loss: 0

In [65]:
modelInceptV3

In [52]:
history2 = model.fit(train_set,epochs = 20, validation_data = test_set)

Train for 64 steps, validate for 64 steps
Epoch 1/20
64/64 [==============================] - 3665s 57s/step - loss: 0.0441 - accuracy: 0.9829 - val_loss: 1.0271 - val_accuracy: 0.7485
Epoch 2/20
64/64 [==============================] - 3675s 57s/step - loss: 0.0437 - accuracy: 0.9861 - val_loss: 1.8969 - val_accuracy: 0.6748
Epoch 3/20
64/64 [==============================] - 3677s 57s/step - loss: 0.0362 - accuracy: 0.9878 - val_loss: 1.3825 - val_accuracy: 0.7090
Epoch 4/20
64/64 [==============================] - 3661s 57s/step - loss: 0.0400 - accuracy: 0.9854 - val_loss: 1.3830 - val_accuracy: 0.7100
Epoch 5/20
64/64 [==============================] - 3662s 57s/step - loss: 0.0361 - accuracy: 0.9885 - val_loss: 1.2236 - val_accuracy: 0.7065
Epoch 6/20
64/64 [==============================] - 3647s 57s/step - loss: 0.0298 - accuracy: 0.9890 - val_loss: 1.2859 - val_accuracy: 0.7373
Epoch 7/20
64/64 [==============================] - 3619s 57s/step - loss: 0.0306 - accuracy: 0.9871

KeyboardInterrupt: 

In [53]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 20,287,553
Trainable params: 9,702,401
Non-trainable params: 10,585,152
_________________________________________________________________


In [67]:
vgg = tf.keras.applications.VGG19(include_top=False, input_shape = (224,224,3))

In [68]:
for i in range(len(vgg.layers) - 10):
    vgg.layers[i].trainable = False

In [ ]:
for layer in vgg.layers:
    print(layer.trainable)

In [70]:
model = Sequential()
model.add(vgg)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu')) 
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_8 ( (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 513       
Total params: 20,287,553
Trainable params: 17,961,985
Non-trainable params: 2,325,568
_________________________________________________________________


In [72]:
history = model.fit(train_set,epochs = 40, validation_data = test_set)

Train for 64 steps, validate for 64 steps
Epoch 1/40
64/64 [==============================] - 4309s 67s/step - loss: 0.6963 - accuracy: 0.5220 - val_loss: 0.6869 - val_accuracy: 0.5000
Epoch 2/40
64/64 [==============================] - 4299s 67s/step - loss: 0.6530 - accuracy: 0.6257 - val_loss: 0.6269 - val_accuracy: 0.6411
Epoch 3/40
64/64 [==============================] - 4441s 69s/step - loss: 0.6220 - accuracy: 0.6707 - val_loss: 0.6305 - val_accuracy: 0.6396
Epoch 4/40
64/64 [==============================] - 4316s 67s/step - loss: 0.5922 - accuracy: 0.6970 - val_loss: 0.5866 - val_accuracy: 0.7021
Epoch 5/40
64/64 [==============================] - 4312s 67s/step - loss: 0.5482 - accuracy: 0.7249 - val_loss: 0.5818 - val_accuracy: 0.7017
Epoch 6/40
64/64 [==============================] - 4593s 72s/step - loss: 0.4966 - accuracy: 0.7661 - val_loss: 0.5508 - val_accuracy: 0.7202
Epoch 7/40
64/64 [==============================] - 4587s 72s/step - loss: 0.4219 - accuracy: 0.8130

KeyboardInterrupt: 

In [25]:
SaveModel(model, model.name, modelDir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ..\models/sequential_3\assets


'..\\models/sequential_3'

In [26]:

model2 = ConvModel1()

In [ ]:
convmod1 = DepthWiseConvModel1()
model1 = convmod1.Create()
model1.summary()

In [30]:

history1 = model1.fit(train_set,steps_per_epoch = 64,
                    epochs = 20, validation_data = test_set,validation_steps = 64)

Train for 64 steps, validate for 64 steps
Epoch 1/20
64/64 [==============================] - 470s 7s/step - loss: 0.6938 - accuracy: 0.5559 - val_loss: 0.6961 - val_accuracy: 0.5000
Epoch 2/20
64/64 [==============================] - 478s 7s/step - loss: 0.6027 - accuracy: 0.6816 - val_loss: 0.7042 - val_accuracy: 0.5000
Epoch 3/20
64/64 [==============================] - 457s 7s/step - loss: 0.5471 - accuracy: 0.7334 - val_loss: 0.7156 - val_accuracy: 0.5000
Epoch 4/20
64/64 [==============================] - 480s 7s/step - loss: 0.4803 - accuracy: 0.7969 - val_loss: 0.7319 - val_accuracy: 0.5000
Epoch 5/20
64/64 [==============================] - 471s 7s/step - loss: 0.4227 - accuracy: 0.8369 - val_loss: 0.7179 - val_accuracy: 0.5000
Epoch 6/20
64/64 [==============================] - 462s 7s/step - loss: 0.3591 - accuracy: 0.8713 - val_loss: 0.7042 - val_accuracy: 0.5000
Epoch 7/20
64/64 [==============================] - 454s 7s/step - loss: 0.2771 - accuracy: 0.9321 - val_loss: 0

In [ ]:
lrs = [0.001, 0.0007, 0.0005, 0.0003, 0.0001]
nn = [10,20,30,40,10000]
jj = 0
convmod = ConvModel1(learn_rate = lrs[jj], units = 1024, L2 = 0.0001)
model = convmod.Create()
for i in range(60):
    print("Step - {}".format(i))
    if i > nn[jj]:
        jj += 1
        model.lr = lrs[jj]
    history = model.fit(train_set,steps_per_epoch = 64,
                        epochs = 1, validation_data = test_set,validation_steps = 64)

In [ ]:
SaveModel(model, model.name, modelDir)

In [ ]:
histories = dict()
for obj in [model2]:
    for unit in [1024]:
        for lr in [0.0001, 0.0005, 0.001]:
            obj.units = unit
            obj.learn_rate = lr
            model = obj.Create()
            history = model.fit(train_set,
                 steps_per_epoch = 64,
                 epochs = 90, validation_data = test_set,validation_steps = 64)
            SaveModel(model, model.name, modelDir)
            histories[model.name] = history    

In [ ]:
model2.Create().summary()

In [ ]:
intercept = StemDecoratorConv(conv_layer_cnt = 2, intercept_module = InterceptionModelV2())
model = intercept.Create()

In [ ]:
hist = model.fit(train_set,
                 steps_per_epoch = 128 / 8,
                 epochs = 1, validation_data = test_set,validation_steps = 2)

In [ ]:
m1 = ConvModel1()
model1 = m1.Create()
model1.summary()

In [ ]:
m2 = DepthWiseConvModel1()
model2 = m2.Create()
model2.summary()